In [1]:
# 📦 라이브러리 불러오기
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [42]:
# 🔧 멀티 인덱스 푸는 함수
def download_single_close_column(ticker, start="2020-01-01", end="2025-04-16"):
    df = yf.download(ticker, start=start, end=end, group_by='ticker')

    if df.empty:
        raise ValueError(f"❌ {ticker} 데이터 없음")

    if isinstance(df.columns, pd.MultiIndex):
        try:
            df_close = df.xs('Close', axis=1, level=1)
        except KeyError:
            raise ValueError(f"❌ 'Close' 레벨 없음: {ticker}")

        if ticker in df_close.columns:
            df = df_close[[ticker]]
        else:
            df = df_close.iloc[:, [0]]

        df.columns = ['Close_' + ticker.replace("=", "")]
    else:
        if 'Close' not in df.columns:
            raise ValueError(f"❌ 'Close' 컬럼 없음: {ticker}")
        df = df[['Close']]
        df.columns = ['Close_' + ticker.replace("=", "")]

    return df

# 🔍 유효한 티커 확인 함수
def is_valid_ticker(ticker, period="30d", min_rows=10):
    try:
        df = yf.download(ticker, period=period, progress=False)
        return not df.empty and "Close" in df.columns and df['Close'].dropna().shape[0] >= min_rows
    except:
        return False

# 📈 예측 모델 함수 (Prophet, LSTM, GRU)
def evaluate_all_models(ticker_name, ticker_symbol):
    df = download_single_close_column(ticker_symbol, start="2020-01-01", end="2025-04-16")
    df = df.dropna()

    df_prophet = pd.DataFrame()
    df_prophet['ds'] = df.index
    df_prophet['y'] = df.iloc[:, 0].astype(float).values.flatten()

    if df_prophet['y'].dropna().shape[0] < 10:
        raise ValueError("❌ y값이 너무 적음")

    scaler = MinMaxScaler()
    y_data = df_prophet['y'].values.reshape(-1, 1)
    scaled_data = scaler.fit_transform(y_data)

    def create_sequences(data, seq_len=60):
        X, y = [], []
        for i in range(seq_len, len(data)):
            X.append(data[i-seq_len:i])
            y.append(data[i])
        return np.array(X), np.array(y)

    X, y_seq = create_sequences(scaled_data, 60)
    if len(X) == 0:
        raise ValueError("❌ 시퀀스 데이터 부족")

    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    model_lstm = Sequential([LSTM(50, input_shape=(60, 1)), Dense(1)])
    model_lstm.compile(optimizer='adam', loss='mse')
    model_lstm.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    model_gru = Sequential([GRU(50, input_shape=(60, 1)), Dense(1)])
    model_gru.compile(optimizer='adam', loss='mse')
    model_gru.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    model_prophet = Prophet()
    model_prophet.fit(df_prophet)
    future = model_prophet.make_future_dataframe(periods=30)
    forecast = model_prophet.predict(future)

    actual = scaler.inverse_transform(y_test)
    pred_lstm = scaler.inverse_transform(model_lstm.predict(X_test))
    pred_gru = scaler.inverse_transform(model_gru.predict(X_test))
    test_dates = df_prophet['ds'][-len(actual):].reset_index(drop=True)
    forecast = forecast.set_index('ds')
    prophet_pred = forecast.loc[test_dates]['yhat'].values

    mae_lstm = mean_absolute_error(actual, pred_lstm)
    rmse_lstm = np.sqrt(mean_squared_error(actual, pred_lstm))
    mae_gru = mean_absolute_error(actual, pred_gru)
    rmse_gru = np.sqrt(mean_squared_error(actual, pred_gru))
    mae_prophet = mean_absolute_error(actual, prophet_pred)
    rmse_prophet = np.sqrt(mean_squared_error(actual, prophet_pred))

    print(f"\n✅ [{ticker_name}] Prediction Error Metrics")
    print(f"LSTM    - MAE: {mae_lstm:.2f}, RMSE: {rmse_lstm:.2f}")
    print(f"GRU     - MAE: {mae_gru:.2f}, RMSE: {rmse_gru:.2f}")
    print(f"Prophet - MAE: {mae_prophet:.2f}, RMSE: {rmse_prophet:.2f}")

    plt.figure(figsize=(14, 5))
    plt.plot(test_dates, actual, label='Actual', color='black')
    plt.plot(test_dates, pred_lstm, label='LSTM', linestyle='--')
    plt.plot(test_dates, pred_gru, label='GRU', linestyle='-.')
    plt.plot(test_dates, prophet_pred, label='Prophet', linestyle=':')
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.title(f"{ticker_name} Price Prediction")
    plt.xlabel("Year")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 🎯 샘플 종목 리스트 (안정적으로 작동하는 예시)
# sample_commodities = {
#     'Gold': 'GC=F',
#     'Silver': 'SI=F',
#     'Crude Oil': 'CL=F'
# }

import requests
from bs4 import BeautifulSoup

# 📡 Yahoo Finance 원자재 티커 크롤링 함수
def get_yahoo_commodity_tickers():
    url = "https://finance.yahoo.com/commodities/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    tickers = {}
    rows = soup.select('table tbody tr')

    for row in rows:
        name_tag = row.find('td', attrs={'aria-label': 'Name'})
        symbol_tag = row.find('td', attrs={'aria-label': 'Symbol'})

        if name_tag and symbol_tag:
            name = name_tag.text.strip()
            symbol = symbol_tag.text.strip()
            tickers[name] = symbol

    return tickers

# ✅ 유효한 종목만 필터링
commodity_tickers = get_yahoo_commodity_tickers()

print("🔍 유효한 종목 검사 중...\n")
for name, symbol in tqdm(commodity_tickers.items()):
    if is_valid_ticker(symbol):
        valid_commodities[name] = symbol
        print(f"✅ {name}: {symbol}")
    if len(valid_commodities) >= 5:  # 너무 많으면 오래 걸리니까 제한
        break

# 최종 사용 대상
sample_commodities = valid_commodities

valid_commodities = {}
# 🔁 예측 반복 실행
for name, symbol in tqdm(sample_commodities.items()):
    print(f"\n🔍 Evaluating: {name} ({symbol})")
    try:
        evaluate_all_models(name, symbol)
    except Exception as e:
        print(f"⚠️ {name} 예측 중 오류 발생: {e}")

🔍 유효한 종목 검사 중...



0it [00:00, ?it/s]
0it [00:00, ?it/s]
